In [2]:
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet,SGDRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np
import textract
import os
import PyPDF2
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import wordninja
import numpy as np
import nltk
from nltk.corpus import stopwords
import re 
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from pyresparser import ResumeParser
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /home/nadhir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
test_data = pd.read_csv("test.csv")
test_files = test_data["CandidateID"]
train_data = pd.read_csv("train.csv")
train_files = train_data["CandidateID"]
train_score = train_data["Match Percentage"]
names = []
#for name in files:
    #names.append(name)q = s_l.reverse()



In [13]:
def read_pdf_files_train(train_files,train_score):
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()

    
    converted_list = []
    cv = CountVectorizer()
    mat = []
    pure_resumes = []
    pure_id = []
    pure_score = []
    for file,score in zip(train_files,train_score):
        path = f'trainResumes/{file}.pdf'
        #path = path.replace("(","")
        #path = path.replace(")","")
        #path = path.replace(",","")
        #path = path.replace("'","")
        text = textract.process(path,method='pdfminer')
        text = text.decode()

        text = text.rstrip('\r\n')
        
        
        for element in text:
            converted_list.append(element.strip())

        e = "".join(converted_list)
        #print(e)


        a = wordninja.split(e)
        a = " ".join(a)
        e = a.lower()
        e = " ".join([word for word in e.split() if word not in stop_words])
        text_encode = e.encode(encoding="ascii", errors="ignore")
        text_decode = text_encode.decode()
        clean_text = " ".join([word for word in text_decode.split()])
        
        text = re.sub("@\S+", "", clean_text)
        text = re.sub("\$", "", text)
        text = re.sub("https?:\/\/.*[\r\n]*", "", text)
        text = re.sub("#", "", text)
        punct = set(string.punctuation) 
        text = "".join([ch for ch in text if ch not in punct])
        
        words = text.split(" ")
        texts = []
        texts.clear()
        for word in words: 
            w = stemmer.stem(word)
            texts.append(w)
        
        text = " ".join(texts)
        
        
        pure_resumes.append(text)
        pure_id.append(file)
        pure_score.append(score)

        converted_list.clear()    


    
    
    return pure_id,pure_resumes,pure_score






def read_pdf_files_test(test_files):
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    
    converted_list = []
    cv = CountVectorizer()
    mat = []
    pure_resumes = []
    pure_id = []
    

    for file in zip(test_files):
        path = f'testResumes/{file}.pdf'
        path = path.replace("(","")
        path = path.replace(")","")
        path = path.replace(",","")
        path = path.replace("'","")
        text = textract.process(f'{path}',method='pdfminer')
        text = text.decode()

        text = text.rstrip('\r\n')

        
        for element in text:
            converted_list.append(element.strip())

        e = "".join(converted_list)
        #print(e)


        a = wordninja.split(e)
        a = " ".join(a)
        e = a.lower()
        e = " ".join([word for word in e.split() if word not in stop_words])
        
        text_encode = e.encode(encoding="ascii", errors="ignore")
        text_decode = text_encode.decode()
        clean_text = " ".join([word for word in text_decode.split()])
        text = re.sub("@\S+", "", clean_text)
        text = re.sub("\$", "", text)
        text = re.sub("https?:\/\/.*[\r\n]*", "", text)
        text = re.sub("#", "", text)
        punct = set(string.punctuation) 
        text = "".join([ch for ch in text if ch not in punct])
        
        words = text.split(" ")
        texts = []
        texts.clear()
        for word in words: 
            w = stemmer.stem(word)
            texts.append(w)
        text = " ".join(texts)
        
        
        
        pure_resumes.append(text)
        pure_id.append(file)
        converted_list.clear()    


    
    
    return pure_id,pure_resumes






In [15]:
train_ids,train_resumes,train_scores = read_pdf_files_train(train_files,train_score)

In [16]:
test_ids,test_resumes = read_pdf_files_test(test_files)

In [98]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x = vectorizer.fit_transform(train_resumes)

y = vectorizer.transform(test_resumes)
features = vectorizer.get_feature_names()
x_train= x.toarray()

y_test = y.toarray()

In [126]:
training_dataframe = pd.DataFrame(x_train)
training_dataframe.columns = features


In [127]:
testing_data_frame = pd.DataFrame(y_test)
testing_data_frame.columns = features
testing_data_frame.shape

(60, 1799)

In [128]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(train_score)

sel = SelectFromModel(ElasticNet(alpha=1.2,random_state=10))
sel.fit(training_dataframe, encoded)

sel.get_support()
selected_feat= training_dataframe.columns[(sel.get_support())]

training_d = training_dataframe[selected_feat]



sel1= SelectFromModel(SGDRegressor())
sel1.fit(training_d, encoded)

sel1.get_support()
selected_feat= training_d.columns[(sel1.get_support())]

print(selected_feat)

#best Lasso(),Elastic net(),SGDregressor()

Index(['alway', 'apr', 'attent', 'award', 'busi', 'civil', 'cloud', 'comput',
       'creat', 'current', 'data', 'de', 'decis', 'driven', 'ece', 'elast',
       'explor', 'fresher', 'hand', 'improv', 'junior', 'learn', 'look',
       'model', 'network', 'nlp', 'part', 'pattern', 'problem', 'process',
       'set', 'solut', 'statist', 'technolog', 'time', 'understand', 'vision'],
      dtype='object')


In [129]:
print(len(selected_feat))

37


In [101]:
# i get this words from above code

c = ['abil', 'alway', 'apr', 'award', 'busi', 'civil', 'cloud', 'comput',
       'creat', 'current', 'data', 'de', 'decis', 'driven', 'ece', 'elast',
       'execut', 'explor', 'fresher', 'hand', 'improv', 'junior', 'learn',
       'look', 'method', 'model', 'nlp', 'object', 'part', 'pattern',
       'problem', 'process', 'python', 'scienc', 'set', 'solut', 'technolog',
       'time', 'understand', 'vision']


In [102]:
training_dataframe = training_dataframe[c]
testing_data_frame = testing_data_frame[c]

In [114]:
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor,XGBRFRegressor

x = np.array(x_train)


# Training of the regression model

model1 = SGDRegressor()
model1.fit(training_dataframe,train_score)

model2 = LinearSVR(epsilon=0.5)
model2.fit(training_dataframe,train_score)

model3 = ElasticNet()
model3.fit(training_dataframe,train_score)
#good models==linearSVR,Leanear regession,SGDRegressor()

#from sklearn.metrics import r2_score


#print(accuracy_score(y_pred,y_test))
#print(classification_report(y_test, y_pred))
model1_pre = model1.predict(training_dataframe)
model2_pre = model2.predict(training_dataframe)
model3_pre = model3.predict(training_dataframe)

In [115]:
print(r2_score(train_score,model1_pre),r2_score(train_score,model2_pre),r2_score(train_score,model3_pre))
#0.8276572398355251  accuracy = 89.951

0.8200069328951431 0.8116719484329183 0.6758667235909476


In [116]:
prediction1 = model1.predict(testing_data_frame)
prediction2 = model2.predict(testing_data_frame)
prediction3 = model3.predict(testing_data_frame)
#a = pd.DataFrame(prediction,columns=["Match Percentage"])
#final = pd.concat([test_files,a],axis=1)
#final = final.to_csv("submission.csv",index=False)


In [119]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# torch.manual_seed(1)    # reproducible

x = np.array([pd.DataFrame(list(zip(model1_pre,model1_pre,model3_pre)),columns=["prediction1","predicition2","prediction3"])], dtype=np.float32)  # x data (tensor), shape=(100, 1)
y = np.array([pd.DataFrame(train_score)], dtype=np.float32)               # noisy y data (tensor), shape=(100, 1)

x = torch.tensor(x)
y = torch.tensor(y)
# torch can only train on Variable, so convert them to Variable
# The code below is deprecated in Pytorch 0.4. Now, autograd directly supports tensors
# x, y = Variable(x), Variable(y)


class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden1,n_hidden2, n_output):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden1)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden1, n_hidden2)
        self.predict = torch.nn.Linear(n_hidden2, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden1(x)) # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x



In [120]:
net = Net(n_feature=3, n_hidden1=20,n_hidden2=10, n_output=1)     # define the network
print(net)  # net architecture

optimizer = torch.optim.Adam(net.parameters(), lr=0.02) # change the learning rate and get high accuracy
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


for t in range(1500):
    t = t+1
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

    if t % 10 == 0:
        print(f"no epochs : {t} and loss : {loss}")


Net(
  (hidden1): Linear(in_features=3, out_features=20, bias=True)
  (hidden2): Linear(in_features=20, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=1, bias=True)
)
no epochs : 10 and loss : 91.8754653930664
no epochs : 20 and loss : 59.34037399291992
no epochs : 30 and loss : 46.796180725097656
no epochs : 40 and loss : 47.335166931152344
no epochs : 50 and loss : 48.252262115478516
no epochs : 60 and loss : 46.70525360107422
no epochs : 70 and loss : 45.392784118652344
no epochs : 80 and loss : 45.142513275146484
no epochs : 90 and loss : 44.84651184082031
no epochs : 100 and loss : 44.51392364501953
no epochs : 110 and loss : 44.180667877197266
no epochs : 120 and loss : 43.79060745239258
no epochs : 130 and loss : 43.48969650268555
no epochs : 140 and loss : 43.2542724609375
no epochs : 150 and loss : 43.0445671081543
no epochs : 160 and loss : 42.845829010009766
no epochs : 170 and loss : 42.65570068359375
no epochs : 180 and loss : 42.4796104431152

In [121]:
x_test = np.array([pd.DataFrame(list(zip(prediction1,prediction2,prediction3)),columns=["prediction1","prediction2","prediction3"])], dtype=np.float32)
x_test = torch.tensor(x_test)
prediction = net(x_test)

In [122]:
prediction = prediction.detach()

In [123]:
prediction = prediction.numpy()
# first 88.71,88.75

In [124]:
#prediction = model1.predict(testing_data_frame)
a = pd.DataFrame(prediction.reshape(-1,1),columns=["Match Percentage"])
final = pd.concat([test_files,a],axis=1)
final = final.to_csv("submission.csv",index=False)
